In [1]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

In [2]:
tqdm.pandas()
pd.set_option("display.max_rows", 1000)

/home/rexhaif/Projects/RussianObnulenie2020/.venv/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
data = pd.read_csv("uiks-utf8.csv")
data['pct_turnout'] = (data.found_ballots / data.registered_voters) * 100
data['pct_yes']     = (data.yes_votes / data.found_ballots) * 100

In [4]:
data.head(2)

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes,latitude,longitude,pct_turnout,pct_yes
0,Республика Адыгея (Адыгея),1 Адыгейская,УИК №1,2277,1925,1925,0,1749,176,NaN,NaN,84.541063,90.857143
1,Республика Адыгея (Адыгея),1 Адыгейская,УИК №2,2694,2290,2290,7,2117,166,44.983019,39.103872,85.003712,92.445415


In [5]:
tol = 1e-2
indicators = {
    'CloseDivisable@5': lambda x: int(np.isclose([x.pct_yes % 5.0],[0], atol=tol) or np.isclose([x.pct_turnout % 5.0], [0], atol=tol)),
    'CloseDivisable@1': lambda x: int((np.isclose([x.pct_yes % 1.0],[0], atol=tol) or np.isclose([x.pct_turnout % 1.0], [0], atol=tol)) and (x.used_ballots % 10.0 == 0 or x.yes_votes % 10.0 == 0))
}

In [6]:
for key, fn in indicators.items():
    data[key] = data.progress_apply(fn, axis=1)

In [7]:
data['AnyDivisable'] = data.progress_apply(lambda x: np.max([x['CloseDivisable@5'], x['CloseDivisable@1']]), axis=1)

In [17]:
fraud_full = data[data['CloseDivisable@5'] == 1]

In [18]:
print(f"Всего участков в ровными процентами: {len(fraud_full)}")

Всего участков в ровными процентами: 4282


In [19]:
print(f"Сумма проголосовавших на таких участках: {fraud_full['found_ballots'].sum()}")

Сумма проголосовавших на таких участках: 1598473


In [24]:
print(f"{(fraud_full['found_ballots'].sum() / data['found_ballots'].sum()) * 100:.2f}% от общего числа")

2.19% от общего числа


In [23]:
print(f"Процентное соотношение голосов за\против: " +
      f"{fraud_full['pct_yes'].mean():.2f}% ЗА, " + 
      f"{((fraud_full.no_votes / data.found_ballots) * 100).mean():.2f}% Против, " + 
      f"{((fraud_full.incorrect_ballots / fraud_full.found_ballots) * 100).mean():.2f}% Испорченных")

Процентное соотношение голосов за\против: 74.94% ЗА, 24.31% Против, 0.75% Испорченных


In [27]:
len(fraud_full[fraud_full.region_name == 'Нижегородская область']) / len(data[data.region_name == "Нижегородская область"])

0.10359408033826638

In [28]:
len(fraud_full[fraud_full.region_name == 'Территория за пределами РФ']) / len(data[data.region_name == "Территория за пределами РФ"])

0.7559055118110236

In [25]:
fraud_full.region_name.value_counts()

Нижегородская область                       245
Приморский край                             228
Территория за пределами РФ                  192
Сахалинская область                         145
Мурманская область                          144
Ставропольский край                         121
город Санкт-Петербург                       120
Республика Дагестан                         117
Красноярский край                           117
Республика Башкортостан                     116
город Москва                                106
Камчатский край                             105
Московская область                          102
Республика Татарстан (Татарстан)             98
Республика Бурятия                           91
Краснодарский край                           86
Архангельская область                        83
Иркутская область                            79
Свердловская область                         76
Омская область                               69
Хабаровский край                        

In [32]:
fraud_full[fraud_full.tik_name == "39 Нижний Новгород, Нижегородская"]['found_ballots'].value_counts()

11    82
12    46
10    31
9     17
13     4
8      2
Name: found_ballots, dtype: int64

In [10]:
fraud = data[data.registered_voters > 500][data['CloseDivisable@5'] == 1]

/home/rexhaif/Projects/RussianObnulenie2020/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [13]:
print(f"Всего участков с процентом явки\голосов за, делящимся на 5: {len(fraud)}")

Всего участков с процентом явки\голосов за, делящимся на 5: 1126


In [14]:
fraud.region_name.value_counts()

Московская область                          68
Краснодарский край                          55
город Москва                                48
город Санкт-Петербург                       47
Республика Башкортостан                     46
Территория за пределами РФ                  44
Ставропольский край                         43
Нижегородская область                       34
Ростовская область                          34
Республика Дагестан                         33
Волгоградская область                       27
Воронежская область                         27
Брянская область                            25
Республика Татарстан (Татарстан)            25
Саратовская область                         22
Свердловская область                        22
Пермский край                               20
Приморский край                             20
Челябинская область                         19
Кемеровская область - Кузбасс               19
Ленинградская область                       19
Красноярский 

In [39]:
fraud[fraud.region_name == "город Москва"]

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes,latitude,longitude,pct_turnout,pct_yes,CloseDivisable@5,CloseDivisable@1,AnyDivisable
89512,город Москва,2 район Вешняки,УИК №893,1272,954,954,0,649,305,55.740907,37.832827,75.000000,68.029350,1,0,1
89561,город Москва,3 район Восточное Измайлово,УИК №3669,958,958,958,9,834,115,NaN,NaN,100.000000,87.056367,1,0,1
89572,город Москва,4 район Гольяново,УИК №953,1741,1255,1255,0,1004,251,55.813329,37.804072,72.085009,80.000000,1,0,1
89605,город Москва,4 район Гольяново,УИК №986,1940,1455,1455,3,1164,288,55.818680,37.828892,75.000000,80.000000,1,0,1
89778,город Москва,10 район Новогиреево,УИК №3679,956,956,956,29,791,136,NaN,NaN,100.000000,82.740586,1,0,1
89844,город Москва,13 район Преображенское,УИК №3681,1155,1155,1155,0,1100,55,NaN,NaN,100.000000,95.238095,1,1,1
89906,город Москва,16 район Сокольники,УИК №1267,1663,585,585,2,351,232,55.789775,37.684766,35.177390,60.000000,1,0,1
89918,город Москва,16 район Сокольники,УИК №3691,743,743,743,0,594,149,NaN,NaN,100.000000,79.946164,1,0,1
89923,город Москва,16 район Сокольники,УИК №3696,608,608,608,22,324,262,NaN,NaN,100.000000,53.289474,1,0,1
90058,город Москва,24 район Ново-Переделкино,УИК №2669,2041,1633,1633,16,1303,314,55.642708,37.371560,80.009799,79.791794,1,0,1
